# Bag of Words Meets Bags of Popcorn

[Kaggle Chanllenge](https://www.kaggle.com/c/word2vec-nlp-tutorial)
Use Google's Word2Vec for movie reviews

Deadline: 2019/01/05

In [1]:
from time import time
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression

## Import Cleaned Data

In [2]:
train = pd.read_csv( "clean_labeledTrainData.csv", index_col=0 )

unlabeled_train = pd.read_csv( "clean_unlabeledTrainData.csv", index_col=0 )

test = pd.read_csv( "clean_testData.csv", index_col=0 )

In [24]:
train.head()

,review,id,score,sentiment
0,stuff going moment mj started listening music ...,"""5814_8""",8,1
1,classic war worlds timothy hines entertaining ...,"""2381_9""",9,1
2,film starts manager nicholas bell giving welco...,"""7759_3""",3,0
3,must assumed praised film greatest filmed oper...,"""3630_4""",4,0
4,superbly trashy wondrously unpretentious explo...,"""9495_8""",8,1


In [101]:
count_vectorizer = CountVectorizer(max_features=2500, ngram_range=(1, 2))

x_train = count_vectorizer.fit_transform(train.review.tolist()).toarray()
x_test = count_vectorizer.transform(test.review.tolist()).toarray()

### Logistic Regression

In [96]:
lr = LogisticRegression(C=0.2, dual=True)
lr.fit(x_train, train.sentiment.tolist())

result = lr.predict(x_test)

/Users/wulingying/.pyenv/versions/3.7.1/envs/nlp/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/wulingying/.pyenv/versions/3.7.1/envs/nlp/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Random Forest

In [102]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators = 100)
forest = forest.fit(x_train, train.sentiment.tolist())

result = forest.predict(x_test)

### Linear SVC

In [105]:
from sklearn.svm import SVC, LinearSVC

linear_svc = LinearSVC()
linear_svc.fit(x_train, train.sentiment.tolist())

result = linear_svc.predict(x_test)

/Users/wulingying/.pyenv/versions/3.7.1/envs/nlp/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


### Multinomial NB

In [108]:
from sklearn.naive_bayes import MultinomialNB

multi_NB = MultinomialNB()
multi_NB.fit(x_train, train.sentiment.tolist())

result = multi_NB.predict(x_test)

In [109]:
output = pd.DataFrame(data = {"id": test['id'], "sentiment": result})

In [110]:
#output.to_csv("Results/Bag_of_Words_model_feature3k_bigram.csv", index=False, quoting=3)

### Check Model Accuracy

In [99]:
csv = pd.read_csv("Results/Bag_of_Words_model_feature3k_bigram.csv")

In [111]:
def classified_correct(model, i, inside=False):
    if inside:
        true_positive = model["sentiment"][i] == 0 and int(model["id"][i].split('"')[1].split("_")[1]) <= 5
        true_negative = model["sentiment"][i] == 1 and int(model["id"][i].split('"')[1].split("_")[1]) > 5
    
    else:
        true_positive = model["sentiment"][i] == 0 and int(model["id"][i].split("_")[1]) <= 5
        true_negative = model["sentiment"][i] == 1 and int(model["id"][i].split("_")[1]) > 5
    
    return true_positive or true_negative

target = output
correct = np.array([classified_correct(target, i, inside=True) for i in range(target.shape[0])])
print(correct.sum() / target.shape[0])

0.84236


# 